Dataset: https://www.kaggle.com/datasets/wyattowalsh/basketball

In [ ]:
!pip install kaggle
!pip install pysqlite3

import os
import sqlite3
import pandas as pd
import scripts.sql as sql

os.environ["KAGGLE_USERNAME"] = "..."
os.environ["KAGGLE_KEY"] = "..."

!kaggle datasets download -d wyattowalsh/basketball -p dataset
!unzip dataset/basketball.zip -d dataset

In [1]:
import scripts.sql as sql

sql_data_path = "dataset/nba.sqlite"
to_remove = [
    'game_summary',
    'line_score',
    'other_stats',
    'inactive_players',
    'officials'
    'play_by_play',
    'draft_combine_stats',
    'team_info_common'
]

sql.remove_tables(sql_data_path, to_remove)
print_output = sql.print_db_structure(sql_data_path, 2)

with open("structure/kaggle_data_structure.txt", "w") as file:
    file.write(print_output)

Table:
CREATE TABLE "game" (
"season_id" TEXT,
  "team_id_home" TEXT,
  "team_abbreviation_home" TEXT,
  "team_name_home" TEXT,
  "game_id" TEXT,
  "game_date" TIMESTAMP,
  "matchup_home" TEXT,
  "wl_home" TEXT,
  "min" INTEGER,
  "fgm_home" REAL,
  "fga_home" REAL,
  "fg_pct_home" REAL,
  "fg3m_home" REAL,
  "fg3a_home" REAL,
  "fg3_pct_home" REAL,
  "ftm_home" REAL,
  "fta_home" REAL,
  "ft_pct_home" REAL,
  "oreb_home" REAL,
  "dreb_home" REAL,
  "reb_home" REAL,
  "ast_home" REAL,
  "stl_home" REAL,
  "blk_home" REAL,
  "tov_home" REAL,
  "pf_home" REAL,
  "pts_home" REAL,
  "plus_minus_home" INTEGER,
  "video_available_home" INTEGER,
  "team_id_away" TEXT,
  "team_abbreviation_away" TEXT,
  "team_name_away" TEXT,
  "matchup_away" TEXT,
  "wl_away" TEXT,
  "fgm_away" REAL,
  "fga_away" REAL,
  "fg_pct_away" REAL,
  "fg3m_away" REAL,
  "fg3a_away" REAL,
  "fg3_pct_away" REAL,
  "ftm_away" REAL,
  "fta_away" REAL,
  "ft_pct_away" REAL,
  "oreb_away" REAL,
  "dreb_away" REAL,
  "reb_a

In [2]:
data_extracted = [
    ('dataset/NBA_Player_Box_Score_Stats.csv', 'player_box_score'), 
    ('dataset/NBA_Player_Stats.csv', 'player_stats')
]

sql_data_path = "dataset/nba_players.sqlite"
sql.create_db_from_csv(sql_data_path, data_extracted)
print_output = sql.print_db_structure(sql_data_path, 2)

with open("structure/player_data_structure.txt", "w") as file:
    file.write(print_output)

C:\Users\Matteo\anaconda3\lib\site-packages\pandas\core\generic.py:2872: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(


ValueError: Table 'player_box_score' already exists.